IOU与GIOU
===
IOU(Intersection Over Union)在目标检测中是一个非常重要的指标，NMS(Non-Max Supression非极大值抑制)是过滤重复预测框的非常重要的一个办法

# 1.IOU
## 1.1.IOU介绍
在目标检测的评价体系中，有一个参数叫做IoU，简单来讲就是模型产生的目标窗口和原来标记窗口的交叠率。具体我们可以简单的理解为检测结果(DetectionResult)与Ground Truth的交集比上它们的并集，即为检测的准确率IoU
$$IOU=\frac{DR \cap GT}{DR \cup GT}$$
![image](Images/03/02/03_01_001.png)
GT = GroundTruth; DR = DetectionResult; 

黄色边框框起来的是$GT \cap DR$，绿色框框起来的是$GT \cup DR$,当然最理想的情况就是 DR 与 GT 完全重合，即IOU=1
![image](Images/03/02/03_01_002.png)

## 1.2.IOU所引起的问题
![image](Images/03/02/03_01_003.jpg)
IoU 是目标检测领域最重要的评价尺度之一，特性是对尺度不敏感，主要判断检测框的重合程度。但是对于CNN而言，没有方向信息，无法反馈神经网络的边界回归框应该如何调整。IoU作为度量和损失都有两个主要问题
1. 如果两个对象不重叠，IoU值将为零，并且不能反映两个形状之间的距离。在非重叠对象的情况下，如果使用IoU作为损失，则梯度为零，无法优化
2. IoU不能正确区分两个对象的不同对齐。更精确地说，对于两个具有相同交点水平的不同方向重叠的对象，IoU将是完全相等的例如图二，因此，IoU函数的值并不能反映两个对象是如何重叠的。
![image](Images/03/02/03_01_004.jpg)
于是通过将IoU的概念扩展到非重叠的情况来解决IoU的这两个弱点。这个推广遵循三个条件
1. 与IOU相同的定义，即将比较对象的形状属性编码到区域属性中
2. 保持IOU的尺度不变性质
3. 确保在发生重叠物体时与IOU有很强的相关性

# 2.GIOU(Generalized IOU)-目标检测任务的新Loss
对于两个任意凸型(体积)$A,B \subseteq S \in R^n$，首先求出了包含A和B的最小凸型$C \subseteq \in R^n$，为了比较两种特定的几何形状，C可以来自同一类型

我们找到一个最小的封闭形状C，让C可以把A，B包含在内，然后我们计算C中没有覆盖A和B的面积占C总面积的比值，然后用A与B的IoU减去这个比值
![image](Images/03/02/03_01_005.jpg)

## 2.1.GIOU的性质
1. 与IOU类似，GIOU也可以作为一个距离，loss可以用下面的公式计算$L_{GIOU}=1-L_{GIOU}$
2. 同原始IoU类似，GIoU对物体的大小不敏感。GIoU总是小于等于IoU，对于IoU，有:$0 \leq IOU \leq 1$，而GIOU则是$-1 \leq GIOU \leq 1$
3. 由于GIoU引入了包含A，B两个形状的C，所以当A，B不重合时，依然可以进行优化
4. 与IoU相比，GIoU不仅关注重叠区域。封闭形状C中两个对称形状A和B之间的空隙在A和B之间没有很好的对齐时增大,因此GIoU的值更能反映两个对称物体之间是如何重叠的

总之就是保留了IoU的原始性质同时弱化了它的缺点

## 2.2.GIOU作为BBox回归的损失
假设有输入为预测框$B^p=(x)1^p,y_1^p,x_2^p,y_2^p)$和真实框$B_g=(x_1^g,y_1^g,x_2^g,y_2^g)$。输出为$L_{IOU}$和$L_{GIOU}$
1. 对于预测框$B^p$来说，有
$$x_2^p > x_1^p, y_2^p > y_1^p$$
且有
$$\begin{split}
\hat{x}_1^p&=min(x_1^p,x_2^p) \\
\hat{x}_2^p&=max(x_1^p,x_2^p) \\
\hat{y}_1^p&=min(y_1^p,y_2^p) \\
\hat{y}_2^p&=max(y_1^p,y_2^p)
\end{split}$$
2. 计算$B^g$的面积
$$A^g=(x_2^g-x_1^g) \times (y_2^g-y_1^g)$$
3. 计算$B^p$的面积
$$A^p=(x_2^p-x_1^p) \times (y_2^p-y_1^p)$$
4. 计算$B^g,B^p$的重叠面积
假设如下
$$\begin{split}
x_1^I&=max(\hat{x}_1^p,x_1^g) \\
x_2^I&=min(\hat{x}_2^p,x_2^g) \\
y_1^I&=max(\hat{y}_1^p,y_1^g) \\
y_2^I&=max(\hat{y}_2^p,y_2^g)
\end{split}$$
有
$$I=\begin{cases}
(x_2^I-x_1^I) \times (y_2^I-y_1^I) & x_2^I > x_1^I,y_2^i > y_1^I \\
0 & otherwise
\end{cases}$$
5. 找到可以包含$B^p,B^g$的最小的box $B^c$
$$\begin{split}
x_1^c&=min(\hat{x}_1^p,x_1^g) \\
x_2^c&=max(\hat{x}_2^p,x_2^g) \\
y_1^c&=min(\hat{y}_1^p,y_1^g) \\
y_2^c&=max(\hat{y}_2^p,y_2^g)
\end{split}$$
6. 计算$B^c$的面积
$$A^c=(x_2^c-x_1^c) \times (y_2^c-y_1^c)$$
7. 计算IOU
$$IOU=\frac{I}{U}=\frac{I}{A^p+A^g-I}$$
8. 计算GIOU
$$GIOU=IOU-\frac{A^c-U}{A^c}$$
9. 计算损失
$$L_{GIOU}=1-GIOU$$

## 2.3.计算GIOU

In [1]:
import numpy as np
def bbox_overlaps_giou(bboxes1, bboxes2):
    """Calculate the gious between each bbox of bboxes1 and bboxes2.
    Args:
        bboxes1(ndarray): shape (n, 4)
        bboxes2(ndarray): shape (k, 4)
    Returns:
        gious(ndarray): shape (n, k)
    """


    bboxes1 = bboxes1.astype(np.float32)
    bboxes2 = bboxes2.astype(np.float32)
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = np.zeros((rows, cols), dtype=np.float32)
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = np.zeros((cols, rows), dtype=np.float32)
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0] + 1) * (
        bboxes1[:, 3] - bboxes1[:, 1] + 1)
    area2 = (bboxes2[:, 2] - bboxes2[:, 0] + 1) * (
        bboxes2[:, 3] - bboxes2[:, 1] + 1)
    for i in range(bboxes1.shape[0]):
        x_start = np.maximum(bboxes1[i, 0], bboxes2[:, 0])
        x_min = np.minimum(bboxes1[i, 0], bboxes2[:, 0])
        y_start = np.maximum(bboxes1[i, 1], bboxes2[:, 1])
        y_min = np.minimum(bboxes1[i, 1], bboxes2[:, 1])
        x_end = np.minimum(bboxes1[i, 2], bboxes2[:, 2])
        x_max = np.maximum(bboxes1[i, 2], bboxes2[:, 2])
        y_end = np.minimum(bboxes1[i, 3], bboxes2[:, 3])
        y_max = np.maximum(bboxes1[i, 3], bboxes2[:, 3])

        overlap = np.maximum(x_end - x_start + 1, 0) * np.maximum(y_end - y_start + 1, 0)
        closure = np.maximum(x_max - x_min + 1, 0) * np.maximum(y_max - y_min + 1, 0)

        union = area1[i] + area2 - overlap
        closure
        ious[i, :] = overlap / union - (closure - union) / closure
    if exchange:
        ious = ious.T
    return ious

In [2]:
box1 = np.array([[0,0,100,100]]*3)
box2 = np.array([[0,200,100,300]]*3)
print(bbox_overlaps_giou(box1, box2))





[[-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]
 [-0.32890365 -0.32890365 -0.32890365]]
